In [1]:
#创建fp树的节点
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name=nameValue  #节点元素名称  在构建时初始化为给定值
        self.count=numOccur  #出现次数  在构建时初始化为给定值
        self.nodeLink=None  #指向下一个相似节点的指针  默认为None
        self.parent=parentNode  #指向父亲节点的指针  在构建时初始化为给定值
        self.children={}  #指向子节点的字典，以节点的元素名称为链  指向子节点的指针为值  初始化为空字典
    
    def increte(self,numOccur):
        self.count += numOccur
    
    def display(self,level=1):
        print('  '*level,self.name,'\t',self.count)
        for child in self.children.values():
            child.display(level+1)

In [2]:
#测试
rootNode=treeNode('fruit',9,None)
rootNode.children['apple']=treeNode('apple',13,None)
rootNode.children['pear']=treeNode('pear',3,None)
rootNode.display()

   fruit 	 9
     apple 	 13
     pear 	 3


 输入：数据集，最小支持度  
 输出  FP树  头指针表
  1.遍历数据集，统计各元素出现次数  创建头指针
  2.移除头指针中不满足最小尺度的元素项
    3.第二次遍历数据集，创建FP数，对每个数据集 中的项集：
       3.1初始化空FP树
       3.2对每个项集进行过滤与重排序
       3.3使用这个项集更新FP树，从FP树的根节点开始：
         3.3.1 如果当前项集的第一个元素项存在于FP树当前的子节点中，则更新这个子节点的计数值
         3.3.2 否则创建新的子节点，更新头指针表
         3.3.3 对当前项集的其余元素项和当前元素项对应的子节点递归3.3过程

In [ ]:
'''
功能 将 新的相似的节点添加到  头指针对应元素的链表结构的最后
参数说明：
    nodeToUpdate：头指针中对应的节点，将 targetNode 添加到它的链表的最后一个元素上  nodeLink上
    targetNode  添加相似的节点
'''
def updateHeader(nodeToUpdate,targetNode):
    #从头指针表nodeLink开始，一直沿着nodeLink直到链表的最后
    while(nodeToUpdate.nodeLink !=None):
        nodeToUpdate=nodeToUpdate.nodeLink
    #在最后加入的最新的相似节点
    nodeToUpdate.nodeLink=targetNode

In [4]:
'''
更新FP树
参数说明：
    orderedItems  对应一条事务经过筛选后的路径  只保留了频繁项 并且排好了序
    rootTree   需更新树的节点
'''
def updateTree(orderedItems,rootTree,headerTable,count):
    #print(orderedItems,'')
    #1.路径：筛选后的事务第一个元素是否已经是  rootTree的子节点  如果是，则增加数量
    if orderedItems[0] in rootTree.children:
        #有该元素项 计数值加1
        rootTree.children[orderedItems[0]].increte(count)
    else:
        # 2. 没有这个元素项时创建一个新的节点  并且添加到树中 同时还要更新头指针表  以指向新的节点
        rootTree.children[orderedItems[0] ]=treeNode(orderedItems[0],count,rootTree)
        #  更新头指针或前一个相似元素节点的指针指向新的节点
        if headerTable[orderedItems[0] ][1]==None: #如果 headerTable中 键为 orderedItems[0]  对应的值[次数,指针]的指针为空，则添加这个指针
            headerTable[orderedItems[0]][1]=rootTree.children[  orderedItems[0]  ]
        else:  #如果指针已经有一个   则需要重新更新  头指针表 用新的   rootTree.children[ orderedItems[0] ] 去更新 headerTable[orderedItems[0]]
            updateHeader( headerTable[orderedItems[0]][1],rootTree.children[orderedItems[0] ] )
            
    # orderedItems中如果还有节点  则不断迭代调用自身  每次调用去掉列表中的第一个元素  递归更新树
    if len(orderedItems)>1:
        #对剩下的元素项迭代调用updateTree函数
        #orderedItems[1::] 切片 将首个节点去掉
        #rootTree.children[orderedItems[0]]  :在fp树中 找到  orderedItems[0] 这个点
        updateTree(orderedItems[1::],rootTree.children[orderedItems[0]],headerTable,count)

In [5]:
'''
创建树
    参数说明：
        dataset：原始事务集
            {frozenset({'z'}):1,
             frozenset({'h','j','p','r','z'}):1,
             ...
            }
        minSup:最小支持度
    返回 FP树  头指针表
'''
def createTree(dataSet,minSup=1):
    '''创建 FP树 '''
    #第一次遍历数据集   创建头指针表
    headerTable = {}  #第一次循环产生结构:  {k-1频繁项 : 次数}
    for transaction in dataSet:
        for item in transaction:
            headerTable[item] = headerTable.get(item,0) + dataSet[transaction]
    #移除不满足最小支持度的元素项
    '''
    原python2可以  但 python3  字典在遍历时不能进行修改，建议转成列表或集合处理    
    '''
    for key in list(headerTable.keys()):
        if headerTable[key]<minSup:
            del(headerTable[key])
    
    #存所有的频繁项的集合
    freqItemSet = set(headerTable.keys())
    #空频繁元素集  返回空
    if len(freqItemSet)==0:
        return None,None
    #增加一个数据项，用于存放指向相似元素项指针  {k-1频繁项:[次数,指针] }
    for key in headerTable:
        headerTable[key] = [headerTable[key],None]
    rootTree=treeNode('Null Set',1,None)  #根节点  是一个空元素
    #第二次遍历数据集  对每一个事务  创建一个FP树的一条路径
    for transaction,count in dataSet.items():
        # localD 中存的是一条路径（对应一个transaction）
        localD={}  #对项集  transet  记录其所有元素的全局频率  用于排序    数据结构：{频繁项:次数}
        #循环事务中的每一个item 判断是否为频繁项  如果是 添加到localD中
        for item in transaction:
            if item in freqItemSet:
                #{'r':5,'z':4}
                localD[item]=headerTable[item][0]
        if len(localD)>0:
            #localD.item()  -》  返回一个元祖  （键,值）的集合  p就是一个键值对的元祖 p[1]取元祖的第二个值
            #根据 localD每个键值对进行排序
            # orderedItems 排序后的频繁项列表 [项，项...]
            orderedItems = [v[0] for v in sorted(localD.items(),key=lambda p:p[1],reverse=True)]#得到排好序的一条
            updateTree(orderedItems,rootTree,headerTable,count) 
    return rootTree,headerTable

In [6]:
#测试以上函数

#  定一个函数：加载数据集
def loadSimpleData():
    simpleData=[['r','z','h','j','p'],
                ['z','y','x','w','v','u','t','s'],
                ['z'],
                ['r','x','n','o','s'],
                ['y','r','x','z','q','t','p'],
                ['y','z','x','e','q','s','t','m']]
    return simpleData

#定义一个函数  用于将事务列表转为frozenset，并存为字典，每条记录记值为1
def createInitSet(dataSet):
    retDict={}
    for trans in dataSet:
        retDict[frozenset(trans)]=1
    return retDict

In [7]:
dataSet=loadSimpleData()
print(dataSet)

[['r', 'z', 'h', 'j', 'p'], ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'], ['z'], ['r', 'x', 'n', 'o', 's'], ['y', 'r', 'x', 'z', 'q', 't', 'p'], ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]


In [8]:
transactionDict=createInitSet(dataSet)
transactionDict

{frozenset({'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [9]:
rootTree,headTable=createTree(transactionDict,3)
rootTree.display()
print(headTable)

   Null Set 	 1
     z 	 5
       r 	 1
       x 	 3
         t 	 3
           s 	 2
             y 	 2
           y 	 1
             r 	 1
     x 	 1
       s 	 1
         r 	 1
{'z': [5, <__main__.treeNode object at 0x00000142E463EA58>], 'r': [3, <__main__.treeNode object at 0x00000142E463E668>], 't': [3, <__main__.treeNode object at 0x00000142E463E898>], 'x': [4, <__main__.treeNode object at 0x00000142E463E908>], 's': [3, <__main__.treeNode object at 0x00000142E463EA20>], 'y': [3, <__main__.treeNode object at 0x00000142E463EA90>]}


# 从一颗FP树中挖掘频繁项集
从FP树抽取频繁项集的三个基本步骤如下：


#### 1.从FP树中获得条件模式基
2.利用条件模式基，构建一个条件FP树
#### 3.迭代重复步骤1步骤2，直到树包含一个元素项为止.

##### 1.抽取条件模式基什么是条件模式基，条件模式基是以所查找元素项为结尾的路径集合，每一条路径其实都是一条前缀路径（prefix path）  简而言之，一条前缀
路径介于所查找的元素项与树根节点之间的所有内容

从上面发现已经保存在头指针的表中的单个频繁元素开始 对于每一个元素项 获得其对应的条件模式基

In [10]:
'''
给定元素项生成一个条件模式基(前缀路径)  
baseRequenceList:输入的频繁项
treeNode  当前FP树的第一个节点 
condPats  ： {frozenset(前缀路径):计数值}
'''
def findPrefixPath(baseRequenceList,treeNode):
    #创建前缀路径
    condPats={}
    while treeNode !=None:
        prefixPath=[]
        ascendTree(treeNode,prefixPath)  #向这个节点的前面查找  形成路径
        if len(prefixPath)>1 :
            # prefixPath 下标为0的是第一个  TreeNode  不用取出，所以从1开始才是 条件模式基
            condPats[frozenset(prefixPath[1:])]=treeNode.count
        treeNode=treeNode.nodeLink
    return condPats

'''
函数功能：直接修改prefixPath的值 将当前节点leafNode添加到prefixPath的末尾，然后递归添加到其父节点。
返回值:prefixPath就是一条从 treeNode（包括）到根节点（不包括）的路径
调用方法:主函数findPrefixPath() 中取出 prefixPath[1:]  即为treeNode的前缀路径
'''
def ascendTree(leafNode,prefixPath):
    if leafNode.parent !=None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent,prefixPath)

In [12]:
print(findPrefixPath('x',headTable['x'][1] ) )    #  headerTable   {k-1频繁项:[次数,指针]}

{frozenset({'z'}): 3}


# 2.创建条件FP树
对于每一个频繁项，都要创建一个条件FP树 可以使刚才发现的条件模式基作为输入数据，并且通过相同的建树代码构建这些树过程说明
##### 比如  t的条件树FP树
###        它的条件模式基
###        最小支持度 3
###        则可以过滤掉  r
###        再形成FP树
            x:3
            y:3
            z:3

In [14]:
#分析一下代码  用于根据 每一个频繁项  basePat 先找到条件模式基 ，再创建一颗条件FP树
condPattBases=findPrefixPath('x',headTable['x'][1] )
myCondTree,myHead=createTree(condPattBases,2)
print('根据条件模式基生成的条件FP树:',myCondTree)
myCondTree.display()
print('这个条件FP树对应的头指针表:',myHead)

根据条件模式基生成的条件FP树: <__main__.treeNode object at 0x00000142E452ABE0>
   Null Set 	 1
     z 	 3
这个条件FP树对应的头指针表: {'z': [3, <__main__.treeNode object at 0x00000142E452A898>]}


In [15]:
condPattBases=findPrefixPath('s',headTable['s'][1])
myCondTree,myHead=createTree(condPattBases,2)
print('根据条件模式基生成的条件FP树：',myCondTree)
myCondTree.display()
print('这个条件FP树对应的头指针表{0}'.format(myHead))

根据条件模式基生成的条件FP树： <__main__.treeNode object at 0x00000142E4638F98>
   Null Set 	 1
     x 	 3
       t 	 2
         z 	 2
这个条件FP树对应的头指针表{'t': [2, <__main__.treeNode object at 0x00000142E4637EF0>], 'z': [2, <__main__.treeNode object at 0x00000142E4637128>], 'x': [3, <__main__.treeNode object at 0x00000142E4638588>]}


# 3.递归查找频繁项集
  有了FP树和条件FP树，我们就可以在前两步的基础上递归得查找频繁项集
    
    递归过程分析如下：
    输入我们有当前数据集的FP树（rootTree,headerTable）
    1.初始化一个空的列表prefix表示前缀
    2.初始化一个空的列表freqItemList接受生成的频繁项集（作为输出）
    3.对headerTable中的每个元素basePat（按计数值由小到大），递归：
        3.1 记basePat+preFix为当前频繁项集newFreqSet
        3.2 将newFreqSet添加到freqItemList中
        3.3 计算newFreqSet的条件FP树 (myConditionTree,myHeaderTable)
        3.4 当条件FP树不为空时，继续下一步：否则退出递归
        3.5 myConditionTree,myHeaderTable  以newFreqSet为新的prefix 

In [16]:
'''
输入参数：
    rootTree和headtable是由createTree()函数生成的数据集的FP树
    minSup表示最小支持度
    prefix请传入一个空集合(set([])),将在函数中用于保存当前的前缀
    freqItemList请传入一个空列表([])  将用来储存生成的频繁项集
'''
def mineTree(rootTree,headTable,minSup,preFix,freqItemList):
    bigL=[v[0] for v in sorted(headTable.items(),key=lambda p:str(p[1]) )]
    #从底层（即次数少的频繁项）开始循环
    for basePat in bigL:
        #加入频繁项列表
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        
        #递归调用函数来创建条件模式基
        conditionPatternBases = findPrefixPath(basePat,headTable[basePat][1])
        #构建条件模式Tree 及头指针列表
        myCondTree,myHead = createTree(conditionPatternBases,minSup)
        
        if myHead !=None:
            print('conditional pattern tree for:{0}'.format(newFreqSet))
            myCondTree.display()
            
            mineTree(myCondTree,myHead,minSup,newFreqSet,freqItemList)

In [17]:
bigL=[v[0] for v in sorted(headTable.items(),key=lambda p: str(p[1] ) )]
bigL

['r', 't', 's', 'y', 'x', 'z']

In [18]:
freq=[]
mineTree(rootTree,headTable,3,set([]),freq)

conditional pattern tree for:{'t'}
   Null Set 	 1
     z 	 3
       x 	 3
conditional pattern tree for:{'t', 'x'}
   Null Set 	 1
     z 	 3
conditional pattern tree for:{'s'}
   Null Set 	 1
     x 	 3
conditional pattern tree for:{'y'}
   Null Set 	 1
     t 	 3
       z 	 3
         x 	 3
conditional pattern tree for:{'y', 'x'}
   Null Set 	 1
     t 	 3
       z 	 3
conditional pattern tree for:{'z', 'y', 'x'}
   Null Set 	 1
     t 	 3
conditional pattern tree for:{'z', 'y'}
   Null Set 	 1
     t 	 3
conditional pattern tree for:{'x'}
   Null Set 	 1
     z 	 3


# 新闻网站点击中挖掘新闻报道

In [19]:
parseData=[line.split() for line in open('kosarak.dat').readlines()]
initSet=createInitSet( parseData)
myFPTree,myHeadTable=createTree(initSet,100000)
myFreqList=[]
mineTree(myFPTree,myHeadTable,100000,set([]),myFreqList)
myFreqList

conditional pattern tree for:{'1'}
   Null Set 	 1
     6 	 107404
conditional pattern tree for:{'3'}
   Null Set 	 1
     6 	 186289
       11 	 117401
     11 	 9718
conditional pattern tree for:{'3', '11'}
   Null Set 	 1
     6 	 117401
conditional pattern tree for:{'11'}
   Null Set 	 1
     6 	 261773


[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]